Based on a more detailed PCA reading, I am going to iterate through various combinations for the f3 statistic.

In [1]:
%run ../scripts/notebook_settings.py

In [2]:
meta_data_samples = pd.read_table("../data/metadata_with_x_missing.txt", sep=" ")
meta_data_samples_f = meta_data_samples.loc[meta_data_samples.Sex=="F"]

In [3]:
zarr_dir = "/faststorage/project/baboondiversity/data/PG_panu3_zarr_12_03_2021/callset.zarr/chrX"
#Opening the zarr data
callset = zarr.open_group(zarr_dir, mode="r")
gt = allel.GenotypeArray(callset["calldata/GT"])

In [4]:
origin_list = []
s_list = []
for origin in meta_data_samples.Origin.unique():
    ss = meta_data_samples_f.loc[meta_data_samples_f.Origin == origin]
    if len(ss) > 4:
        origin_list.append(origin)
        s_list.append(ss.Species.unique()[0])

In [5]:
origin_list, s_list

(['Mikumi, Tanzania',
  'Chunga, Zambia',
  'Serengeti, Tanzania',
  'Ngorongoro, Tanzania',
  'Gombe, Tanzania',
  'Lake Manyara, Tanzania',
  'Niokolo-Koba, Senegal',
  'Filoha, Ethiopia',
  'Gog Woreda, Gambella region, Ethiopia'],
 ['cynocephalus',
  'kindae',
  'anubis',
  'anubis',
  'anubis',
  'anubis',
  'papio',
  'hamadryas',
  'anubis'])

In [6]:
def f3_run(recipient, d1, d2):
    r_ids = meta_data_samples_f.loc[meta_data_samples_f.Origin == recipient].callset_index.values
    d1_ids = meta_data_samples_f.loc[meta_data_samples_f.Origin == d1].callset_index.values
    d2_ids =meta_data_samples_f.loc[meta_data_samples_f.Origin == d2].callset_index.values
    r_gt = gt.take(r_ids, axis=1)
    d1_gt = gt.take(d1_ids, axis=1)
    d2_gt = gt.take(d2_ids, axis=1)
    r_ac = r_gt.count_alleles(max_allele=1)
    d1_ac = d1_gt.count_alleles(max_allele=1)
    d2_ac = d2_gt.count_alleles(max_allele=1)
    f3_stats = allel.average_patterson_f3(r_ac, d1_ac, d2_ac, blen=100000)
    return f3_stats
    
f3_run('Ngorongoro, Tanzania', 'Gombe, Tanzania', 'Filoha, Ethiopia')

/home/eriks/miniconda3/envs/baboondiversity/lib/python3.7/site-packages/allel/stats/admixture.py:38: RuntimeWarning: invalid value encountered in true_divide
  x = (ac[:, 0] * ac[:, 1]) / (an * (an - 1))


(0.1762753890232439,
 0.017917753195189654,
 9.838029751998606,
 array([0.03885684, 0.08331648, 0.10474906, 0.19118365, 0.222043  ,
        0.32407926, 0.26872401, 0.13455264, 0.21950206, 0.19122057,
        0.1511306 , 0.07974324, 0.25114002, 0.10869125, 0.15679132,
        0.34730926, 0.31814286, 0.24427994, 0.14601502, 0.26153399,
        0.00249366, 0.04733102, 0.06855495, 0.11039656, 0.30931562,
        0.23348827, 0.21216534, 0.17485718, 0.29984296]),
 array([0.18214896, 0.1797322 , 0.17966288, 0.17732002, 0.17659535,
        0.17353512, 0.17509927, 0.1800852 , 0.17630991, 0.17720952,
        0.17885187, 0.18105781, 0.17537484, 0.17999426, 0.17838646,
        0.17204264, 0.17329725, 0.17463783, 0.17905948, 0.17474422,
        0.18519456, 0.18402982, 0.18298834, 0.17976618, 0.17337706,
        0.17559331, 0.17625154, 0.17778923, 0.1729457 ]))

In [7]:
d1_l = []
d2_l = []
r_l = []
f3_l = []
z_l = []

for i in range(len(origin_list)):
    print(i)
    for j in range(i):
        for x in origin_list:
            if x != origin_list[i] and x != origin_list[j]:
                f3_stats = f3_run(x, origin_list[i], origin_list[j])
                d1_l.append(origin_list[i])
                d2_l.append(origin_list[j])
                r_l.append(x)
                f3_l.append(f3_stats[0])
                z_l.append(f3_stats[2])
d = {"d1": d1_l, "d2": d2_l, "recipient": r_l, "f3": f3_l, "z-score": z_l}
df_f3 = pd.DataFrame(data=d)

0
1
2
3
4
5
6
7
8


In [12]:
df_f3.sort_values(["z-score"])

,d1,d2,recipient,f3,z-score
24,"Ngorongoro, Tanzania","Mikumi, Tanzania","Lake Manyara, Tanzania",0.015969,1.562679
38,"Ngorongoro, Tanzania","Serengeti, Tanzania","Lake Manyara, Tanzania",0.019345,2.208576
66,"Gombe, Tanzania","Ngorongoro, Tanzania","Lake Manyara, Tanzania",0.021316,2.325982
221,"Gog Woreda, Gambella region, Ethiopia","Ngorongoro, Tanzania","Lake Manyara, Tanzania",0.016776,2.356202
130,"Niokolo-Koba, Senegal","Ngorongoro, Tanzania","Lake Manyara, Tanzania",0.015114,2.363432
...,...,...,...,...,...
176,"Filoha, Ethiopia","Gombe, Tanzania","Chunga, Zambia",0.881129,23.048889
6,"Chunga, Zambia","Mikumi, Tanzania","Gog Woreda, Gambella region, Ethiopia",1.201683,23.070706
245,"Gog Woreda, Gambella region, Ethiopia","Filoha, Ethiopia","Mikumi, Tanzania",0.908043,23.136340
162,"Filoha, Ethiopia","Serengeti, Tanzania","Chunga, Zambia",0.899612,23.314581


In [15]:
zarr_dir = "/faststorage/project/baboondiversity/data/PG_panu3_zarr_12_03_2021/callset.zarr/chr7"
#Opening the zarr data
callset = zarr.open_group(zarr_dir, mode="r")
gt = allel.GenotypeArray(callset["calldata/GT"])

In [16]:
d1_l = []
d2_l = []
r_l = []
f3_l = []
z_l = []

for i in range(len(origin_list)):
    print(i)
    for j in range(i):
        for x in origin_list:
            if x != origin_list[i] and x != origin_list[j]:
                f3_stats = f3_run(x, origin_list[i], origin_list[j])
                d1_l.append(origin_list[i])
                d2_l.append(origin_list[j])
                r_l.append(x)
                f3_l.append(f3_stats[0])
                z_l.append(f3_stats[2])
d = {"d1": d1_l, "d2": d2_l, "recipient": r_l, "f3": f3_l, "z-score": z_l}
df_f3_chr7 = pd.DataFrame(data=d)

0
1
2
3
4
5
6
7
8


In [17]:
df_f3_chr7.sort_values(["z-score"])

,d1,d2,recipient,f3,z-score
24,"Ngorongoro, Tanzania","Mikumi, Tanzania","Lake Manyara, Tanzania",0.008658,1.821811
172,"Filoha, Ethiopia","Ngorongoro, Tanzania","Lake Manyara, Tanzania",0.008638,2.246264
31,"Ngorongoro, Tanzania","Chunga, Zambia","Lake Manyara, Tanzania",0.009877,2.428845
130,"Niokolo-Koba, Senegal","Ngorongoro, Tanzania","Lake Manyara, Tanzania",0.008506,2.501922
65,"Gombe, Tanzania","Ngorongoro, Tanzania","Serengeti, Tanzania",0.009181,2.907800
...,...,...,...,...,...
56,"Gombe, Tanzania","Serengeti, Tanzania","Mikumi, Tanzania",0.564763,47.952239
84,"Lake Manyara, Tanzania","Serengeti, Tanzania","Mikumi, Tanzania",0.541669,48.107587
245,"Gog Woreda, Gambella region, Ethiopia","Filoha, Ethiopia","Mikumi, Tanzania",0.435094,48.237544
238,"Gog Woreda, Gambella region, Ethiopia","Niokolo-Koba, Senegal","Mikumi, Tanzania",0.479142,48.856232


In [22]:
meta_data_samples[9:]

,PGDP_ID,Provider_ID,Provider,Genus,Species,Origin,Sex,address,longitude,latitude,callset_index,C_origin,x_missing
9,PD_0067,1043,Roos,Theropithecus,gelada,captive,M,"SDSU Captive Wildlife Research Facility, Brook...",-96.793280,44.334031,9,"Gelada, Captive",0.024995
10,PD_0199,09SNF1101115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",F,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626,10,"Anubis, Tanzania",0.003591
11,PD_0200,11SNF1101115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",F,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626,11,"Anubis, Tanzania",0.004100
12,PD_0201,19SNM1131115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",M,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626,12,"Anubis, Tanzania",0.010000
13,PD_0202,20SNF1131115,Knauf/Chuma/Roos,Papio,anubis,"Serengeti, Tanzania",M,"Serengeti, Mara, Lake Zone, Tanzania",34.742544,-1.996626,13,"Anubis, Tanzania",0.010041
...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,PD_0790,34418_BZ11065,Rogers/Jolly/Phillips-Conroy,Papio,kindae,"Chunga, Zambia",F,"Chunga, Mumbwa District, Central Province, Zambia",26.005210,-15.053557,222,"Kindae, Zambia",0.009835
223,PD_0791,34419_BZ11066,Rogers/Jolly/Phillips-Conroy,Papio,kindae,"Chunga, Zambia",F,"Chunga, Mumbwa District, Central Province, Zambia",26.005210,-15.053557,223,"Kindae, Zambia",0.009672
224,PD_0792,34420_BZ11067,Rogers/Jolly/Phillips-Conroy,Papio,kindae,"Chunga, Zambia",F,"Chunga, Mumbwa District, Central Province, Zambia",26.005210,-15.053557,224,"Kindae, Zambia",0.010119
225,PD_0793,34422_BZ11070,Rogers/Jolly/Phillips-Conroy,Papio,kindae,"Chunga, Zambia",M,"Chunga, Mumbwa District, Central Province, Zambia",26.005210,-15.053557,225,"Kindae, Zambia",0.014323


In [14]:
def f3_run_c_origin(recipient, d1, d2):
    r_ids = meta_data_samples_f.loc[meta_data_samples_f.C_origin == recipient].callset_index.values
    d1_ids = meta_data_samples_f.loc[meta_data_samples_f.C_origin == d1].callset_index.values
    d2_ids =meta_data_samples_f.loc[meta_data_samples_f.C_origin == d2].callset_index.values
    r_gt = gt.take(r_ids, axis=1)
    d1_gt = gt.take(d1_ids, axis=1)
    d2_gt = gt.take(d2_ids, axis=1)
    r_ac = r_gt.count_alleles(max_allele=1)
    d1_ac = d1_gt.count_alleles(max_allele=1)
    d2_ac = d2_gt.count_alleles(max_allele=1)
    f3_stats = allel.average_patterson_f3(r_ac, d1_ac, d2_ac, blen=50000)
    return f3_stats

f3_run_c_origin("Anubis, Tanzania", "Anubis, Ethiopia", "Cynocephalus, Central Tanzania")

(0.04925679243664045,
 0.006165667579691353,
 7.988882274302922,
 array([ 0.11518858,  0.11351072,  0.1717316 ,  0.11906962,  0.00540426,
         0.06318765,  0.05048688,  0.03601107,  0.14615445,  0.17830246,
        -0.01512364,  0.04458574,  0.1088272 ,  0.04379714,  0.10650415,
         0.00582297,  0.05941642,  0.09653711,  0.0571918 ,  0.01179105,
         0.09036609,  0.040967  ,  0.0355546 ,  0.04444815,  0.02657283,
         0.11903065,  0.06848322,  0.13504745,  0.18217121,  0.01465021,
         0.03323993,  0.01721109,  0.03702931,  0.06461401,  0.05981176,
         0.00334352,  0.07315818,  0.01169817,  0.05005456,  0.00267806,
        -0.04465806,  0.0220328 ,  0.04695055, -0.02651893, -0.00198063,
         0.0361565 , -0.05062908,  0.09845525,  0.0305433 ,  0.07295868,
         0.11077047,  0.05848121,  0.07309587,  0.01884376,  0.04322124,
         0.10917254, -0.0043224 ,  0.03765499]),
 array([0.04929399, 0.04895957, 0.0488153 , 0.04932681, 0.0503549 ,
        0.04968